Nama : Mazaya Nova Shabrina 
Email : injeolminara@gmail.com
Domisili Kota : Purbalingga, Jawa Tengah

In [1]:
import pandas as pd

df= pd.read_csv('sample_data/kaggle_movie_train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22579 entries, 0 to 22578
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      22579 non-null  int64 
 1   text    22579 non-null  object
 2   genre   22579 non-null  object
dtypes: int64(1), object(2)
memory usage: 529.3+ KB


In [2]:
df.head()

,id,text,genre
0,0,"eady dead, maybe even wishing he was. INT. 2ND...",thriller
1,2,"t, summa cum laude and all. And I'm about to l...",comedy
2,3,"up Come, I have a surprise.... She takes him ...",drama
3,4,ded by the two detectives. INT. JEFF'S APARTME...,thriller
4,5,"nd dismounts, just as the other children reach...",drama


In [3]:
df = df.drop(columns=['id'])

In [4]:
df['genre'].value_counts()

drama        8873
thriller     6824
comedy       2941
action       2392
sci-fi        613
horror        456
other         270
adventure     147
romance        63
Name: genre, dtype: int64

In [5]:
df.isnull().sum()

text     0
genre    0
dtype: int64

In [6]:
# gunakan drama, comedy, thriller, scifi dan action
df = df[df['genre'].isin(['drama','thriller','action','comedy','sci-fi'])]

df['genre'].value_counts()

drama       8873
thriller    6824
comedy      2941
action      2392
sci-fi       613
Name: genre, dtype: int64

In [7]:
category = pd.get_dummies(df.genre)
df_new = pd.concat([df, category], axis = 1)
df_new = df_new.drop(columns='genre')
df_new.head()

,text,action,comedy,drama,sci-fi,thriller
0,"eady dead, maybe even wishing he was. INT. 2ND...",0,0,0,0,1
1,"t, summa cum laude and all. And I'm about to l...",0,1,0,0,0
2,"up Come, I have a surprise.... She takes him ...",0,0,1,0,0
3,ded by the two detectives. INT. JEFF'S APARTME...,0,0,0,0,1
4,"nd dismounts, just as the other children reach...",0,0,1,0,0


In [8]:
Text = df_new['text'].values
label = df_new[['action', 'comedy', 'drama','sci-fi','thriller']].values

In [9]:
from sklearn.model_selection import train_test_split

Text_train, Text_test, label_train, label_test = train_test_split(Text, 
                                                                  label,
                                                                  test_size=0.2)

In [15]:
# tokenizer dan pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(Text_train)
tokenizer.fit_on_texts(Text_test)

# Metode tokenizer.fit_on_texts() sebaiknya hanya digunakan untuk data train saja.
# Karena jika dilakukan untuk data test atau seluruh data teks yang ada di dataset maka model akan sulit memprediksi teks yang ‘word’-nya tidak ada di dalam dataset. 
#Selain itu metode fit_on_texts pada data train saja dapat menghindari peluang overfitting

seq_train = tokenizer.texts_to_sequences(Text_train)
seq_test = tokenizer.texts_to_sequences(Text_test)

padded_train = pad_sequences(seq_train)
padded_test = pad_sequences(seq_test)


In [16]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_dim = 5000, output_dim = 16),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(5, activation='softmax')

])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# myCallback

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.9):
      print("\nAkurasi telah mencapai > 90%!")
    self.model.stop_training = True

callbacks = myCallback()

In [ ]:
num_epochs = 30 

history = model.fit(padded_train, label_train,epochs= num_epochs, validation_data = (padded_test, label_test), verbose=2)

Epoch 1/30
542/542 - 80s - loss: 1.2594 - accuracy: 0.4243 - val_loss: 1.0578 - val_accuracy: 0.4923 - 80s/epoch - 147ms/step
Epoch 2/30
542/542 - 77s - loss: 0.9854 - accuracy: 0.4925 - val_loss: 0.9609 - val_accuracy: 0.5096 - 77s/epoch - 142ms/step
Epoch 3/30
542/542 - 77s - loss: 0.9281 - accuracy: 0.5068 - val_loss: 1.0483 - val_accuracy: 0.5027 - 77s/epoch - 142ms/step
Epoch 4/30
542/542 - 77s - loss: 0.8749 - accuracy: 0.5192 - val_loss: 0.9739 - val_accuracy: 0.5158 - 77s/epoch - 141ms/step
Epoch 5/30
542/542 - 76s - loss: 0.8495 - accuracy: 0.5232 - val_loss: 0.9964 - val_accuracy: 0.4941 - 76s/epoch - 141ms/step
Epoch 6/30
542/542 - 77s - loss: 0.7844 - accuracy: 0.5806 - val_loss: 0.9879 - val_accuracy: 0.5824 - 77s/epoch - 141ms/step
Epoch 7/30
542/542 - 76s - loss: 0.5669 - accuracy: 0.7537 - val_loss: 0.7330 - val_accuracy: 0.7796 - 76s/epoch - 139ms/step
Epoch 8/30
542/542 - 76s - loss: 0.3787 - accuracy: 0.8617 - val_loss: 0.5819 - val_accuracy: 0.8159 - 76s/epoch - 140

In [ ]:
#plot loss dan akurasi dari model dengan matplotlib dan tensorboard